In [131]:
import math
from pprint import pprint
from math import log
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import matrix
from importlib import reload

for m in (arrays, trees, lists, graphs, search, strings, matrix):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import matrix as mt

print()

In [84]:
def optimum_multiplications(a: List[int]) -> int:
    """
    Given a list of matrix sizes (a[i] x a[i+1])
    return minimum number of operations.
    """

    @lru_cache(None)
    def sub(i, j):
        # Multiplying (i, k) x (k, j) matrices gives: (i, j) matrix.
        # The number of multiplications necessary is: i * k * j
        # For each K we multiply two matrices resulting from recursive calls:
        #   sub(i, k)     => (a[i - 1], a[k])
        #   sub(k + 1, j) => (    a[k], a[j])
        m = lambda k: sub(i, k) + a[i - 1] * a[k] * a[j] + sub(k + 1, j)
        # Determine the optimal split for this range.
        return min(map(m, range(i, j))) if i < j else 0

    return sub(1, len(a) - 1)

In [124]:
A = ord("A")


def optimum_brackets(a: List[int]) -> str:
    """
    Given a list of matrix sizes (a[i] x a[i+1])
    return optimal bracketed multiplication expression
    in the form:  `A(BC)D`
    """

    @lru_cache(None)
    def sub(i, j):
        # Multiplying (i, k) x (k, j) matrices gives: (i, j) matrix.
        # The number of multiplications necessary is: i * k * j
        # For each K we multiply two matrices resulting from recursive calls:
        #   sub(i, k)     => (a[i - 1], a[k])
        #   sub(k + 1, j) => (    a[k], a[j])
        def split(k):
            return sub(i, k)[0] + a[i - 1] * a[k] * a[j] + sub(k + 1, j)[0], k

        # Determine the optimal split for this range.
        return min(map(split, range(i, j))) if i < j else (0, i)

    def par(i, j):
        if i == j:
            return chr(A + i - 1)
        k = sub(i, j)[1]
        return "(" + par(i, k) + par(k + 1, j) + ")"

    return par(1, len(a) - 1)

In [134]:
mt.optimum_brackets([5, 4, 3, 2, 1])

'A(B(CD))'

In [133]:
mt.optimum_brackets([40, 20, 30, 10, 30])

'A(BC)D'

In [86]:
def get(p):
    n = len(p)
    m = [[0] * n for _ in range(n)]
    for L in range(2, n):
        for i in range(1, n - L + 1):
            m[i][i + L - 1] = float("inf")
            for k in range(i, i + L - 1):
                q = m[i][k] + m[k + 1][i + L - 1] + p[i - 1] * p[k] * p[i + L - 1]
                if q < m[i][i + L - 1]:
                    m[i][i + L - 1] = q
    return m[1][n - 1]

In [88]:
def find(s, p):
    arr = []
    ans = 0
    for t in s:
        if t == "(":
            arr.append((-1, -1))
        elif t == ")":
            b = arr.pop()
            a = arr.pop()
            arr.pop()
            arr.append((a[0], b[1]))
            ans += a[0] * a[1] * b[1]
        else:
            arr.append((p[ord(t) - ord("A")], p[ord(t) - ord("A") + 1]))
    return ans

In [94]:
find("((ABC)D)", [1, 2, 3, 4, 5])

64

In [87]:
get([1, 2, 3, 4, 5])

38

In [85]:
optimum_multiplications([1, 2, 3, 4, 5])

38